In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [ ]:
city = "slo"
query = "cal+poly"
max_price = "1500"
url = "https://%s.craigslist.org/search/apa?query=%s&hasPic=1&max_price=%s&availabilityMode=0" % (city, query, max_price)
base_url = "https://%s.craigslist.org" % (city)
page = requests.get(url)
soup = BeautifulSoup(page.text,"html.parser")

In [ ]:
apts = soup.find_all('p', attrs={'class' : "result-info"})

In [ ]:
header = apts[0].find_all('a')
listings = apts[0].find_all('span')

In [ ]:
title = []
price = []
rooms = []
link = []
for i in range(0,len(apts)):
    title.append({'Title' : apts[i].find_all('a')[0].find(text=True)})
    price.append({'Price': apts[i].find_all('span')[3].find(text=True)})
    if apts[i].find_all('span')[4].find(text=True).find('br') != -1:
        rooms.append({'Rooms': apts[i].find_all('span')[4].find(text=True).replace(' ', '').replace('\n','').replace("-","")})
    else:
        rooms.append({'Rooms' : "N/A"})    
    link.append({'Link' : base_url + apts[i].find_all('a')[0]['href']})

In [ ]:
df_t = pd.DataFrame(title)
df_p = pd.DataFrame(price)
df_r = pd.DataFrame(rooms)
df_l = pd.DataFrame(link)
df = pd.concat([df_t, df_p, df_r, df_l], axis=1)
df_html = df.to_html(index=False)

In [ ]:
metadata = pd.read_csv("metadata.csv")
md_list = list(metadata['metadata'])
md_list

In [ ]:
me = md_list[0]
you = md_list[1]
you1 = md_list[2]
you2 = md_list[3]

msg = MIMEMultipart('alternative')
msg['Subject'] = "Daily Apartment Listings"
msg['From'] = me
msg['To'] = you

# Create the body of the message (a plain-text and an HTML version).
text = "Below are today's Craigslist apartments"
html = df_html
part1 = MIMEText(text, 'plain')
part2 = MIMEText(html, 'html')


msg.attach(part1)
msg.attach(part2)
# Send the message via local SMTP server.
mail = smtplib.SMTP('smtp.gmail.com', 587)

mail.ehlo()

mail.starttls()

mail.login(me, md_list[4])
mail.sendmail(me, you, msg.as_string())
mail.sendmail(me, you1, msg.as_string())
mail.sendmail(me, you2, msg.as_string())
mail.quit()